In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_links = pd.read_excel('db_prod.xlsx')
df_links.head()

,id,link,Objectify (yes or no)
0,1,https://www.instagram.com/p/CzTyW00KYzC/,1.0
1,2,https://www.instagram.com/p/C0PzOJxhEv7/,0.0
2,3,https://www.instagram.com/p/CzxRQd0tbV6/,0.0
3,4,https://www.instagram.com/p/CzZIXBVMWdZ/?img_i...,1.0
4,5,https://www.instagram.com/p/CzZIXBVMWdZ/?img_i...,1.0


In [3]:
df_links.dropna(inplace = True)

In [4]:
links = df_links['link']
links

0               https://www.instagram.com/p/CzTyW00KYzC/
1               https://www.instagram.com/p/C0PzOJxhEv7/
2               https://www.instagram.com/p/CzxRQd0tbV6/
3      https://www.instagram.com/p/CzZIXBVMWdZ/?img_i...
4      https://www.instagram.com/p/CzZIXBVMWdZ/?img_i...
                             ...                        
278    https://www.instagram.com/p/Cy3AwyyMlvd/?utm_s...
279    https://www.instagram.com/p/CystloQtc3B/?utm_s...
280    https://www.instagram.com/p/CxnMNwKsphQ/?utm_s...
281    https://www.instagram.com/p/Cu6q25Us_XC/?utm_s...
282    https://www.instagram.com/p/CtjBFvLtulX/?utm_s...
Name: link, Length: 282, dtype: object

In [5]:
import os

# Get the current working directory
current_directory = os.getcwd()
current_directory
# Specify the folder path you want to search for text files in
folder_path = current_directory + '\\downloaded_posts'

# List all files in the folder
files = os.listdir(folder_path)

# Filter out only the text files (files with a .txt extension)
txt_files = [file for file in files if file.endswith('.txt')]
len(txt_files)

259

### Instaloader

In [ ]:
import instaloader
from instaloader.exceptions import BadResponseException

In [ ]:
ids = df_links['id']
# Initialize a dictionary to map custom IDs to corresponding post names
custom_id_to_post_name = {}

# ... [Your existing code for setup and loading the list_downloaded] ...


for custom_id, link in zip(list_ids, tqdm(links)):
    try:
        post_link = link
        post = instaloader.Post.from_shortcode(L.context, post_link.split('/')[-2])

        if post not in list_downloaded:
            # Download the post
            L.download_post(post, target=download_folder)
            list_downloaded.append(post)

            # Update the dictionary with the custom ID and post name
            custom_id_to_post_name[custom_id] = post.shortcode

    except BadResponseException:
        print(f"Could not fetch metadata for {link}. Skipping...")
        continue
    except Exception as e:
        print(f"An error occurred: {e}")
        continue

    time.sleep(2)

In [ ]:
import os
import instaloader
from instaloader.exceptions import BadResponseException
from tqdm import tqdm
import time
import json

list_ids = df_links['id']
links = df_links['link']


# Assuming 'list_ids' and 'links' are already defined
download_folder = 'downloaded_posts'
L = instaloader.Instaloader()

# Ensure the download folder exists
os.makedirs(download_folder, exist_ok=True)

# Initialize or load the custom_id_to_post_name dictionary
custom_id_to_post_name = {}  # Load from file if needed

for custom_id, link in zip(list_ids, tqdm(links)):
    shortcode = link.split('/')[-2]

    try:
        post = instaloader.Post.from_shortcode(L.context, shortcode)

        # Check if the post has already been downloaded
        post_already_downloaded = False
        for filename in os.listdir(download_folder):
            if filename.endswith(".json") and shortcode in filename:
                post_already_downloaded = True
                custom_id_to_post_name[custom_id] = filename
                break

        if not post_already_downloaded:
            # Download the post
            L.download_post(post, target=download_folder)

            # Find the new file and update the mapping
            for filename in os.listdir(download_folder):
                if filename.endswith(".json") and shortcode in filename:
                    custom_id_to_post_name[custom_id] = filename
                    break

    except BadResponseException:
        print(f"Could not fetch metadata for {link}. Skipping...")
        continue
    except Exception as e:
        print(f"An error occurred: {e}")
        continue

    #time.sleep(2)

# Optionally, save the custom_id_to_post_name dictionary to a file for persistence
with open('custom_id_to_post_name_mapping.json', 'w') as file:
    json.dump(custom_id_to_post_name, file)


In [ ]:
custom_id_to_post_name

In [ ]:
# Convert to DataFrame
df_ids = pd.DataFrame(list(id_to_post.items()), columns=['id', 'PostName'])

df_ids.to_excel('posts_id.xlsx')